In [7]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

In [8]:
def avg_last_execution(files_path, drift_detector, generator):
    
    results = []
    for file in files_path:
        #print(file)
        df = pd.read_csv(file)
        results.append(df.iloc[-1]['classifications correct (percent)'])
        #print(df.iloc[-1]['classifications correct (percent)'])
    
    tuple_results = tuple([np.mean(results), np.std(results), drift_detector, generator])
   
    #print(f'avg of all last execution: {np.mean(median)}')
    return tuple_results 

In [9]:
drift_detectors_params = [
                              {"id": "DDM", "drift_name": "DDM", "params": ""},    \
                              {"id": "EDDM", "drift_name": "EDDM", "params": ""},   \
                              {"id": "ADWIN", "drift_name": "ADWINChangeDetector", "params": ""},  \
                              {"id": "ECDD", "drift_name": "EWMAChartDM", "params": ""},   \
                              {"id": "STEPD", "drift_name": "STEPD", "params": ""},  \
                              {"id": "SeqDrift2", "drift_name": "SeqDrift2ChangeDetector", "params": ""},  \
                              {"id": "SEED", "drift_name": "SEEDChangeDetector", "params": ""},   \
                              {"id": "HDDM_A_Test", "drift_name": "HDDM_A_Test", "params": ""},  \
                              {"id": "HDDM_W_Test", "drift_name": "HDDM_W_Test", "params": ""},  \
                              #{"id": "FHDDM", "drift_name": "FHDDM", "params": ""},  \
                              #{"id": "FTDD", "drift_name": "FTDD", "params": ""},   \
                              {"id": "RDDM_30", "drift_name": "RDDM", "params": "-n 30 -w 2 -o 3"}, \
                              {"id": "RDDM","drift_name": "RDDM", "params": ""},   \
                              #{"id": "WSTD","drift_name": "WSTD", "params": ""},
                              {"id": "NoChangeDetector","drift_name": "NoChangeDetectorNaive", "params": ""}
                        ]

## Table Abrupt NaiveBayes 10K

In [10]:
classificator = "bayes.NaiveBayes"
drift_size = 10000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,65.850667,65.881000,66.131333,65.140667,67.370333,67.579667,58.037667,67.824000,66.942333,66.342000,67.127667,65.325667
AGRAW2,86.090333,85.320667,85.579333,83.945000,86.900000,87.019000,78.757333,86.267333,85.672667,86.108333,87.100667,83.690333
LED,74.799000,74.799000,69.895333,74.752667,74.768000,73.824000,32.264333,74.799000,73.567333,56.613000,68.726000,63.457000
MIXED,90.009667,90.240667,89.451000,89.284000,90.871000,90.867333,77.591000,90.713667,90.353333,89.619667,90.873333,83.803667
RANDOM_RBF,85.669333,86.762000,84.951000,85.523667,86.969000,87.023000,49.996333,86.901667,86.686333,71.150333,84.733333,75.856667
SINE,88.195667,85.708333,88.205667,87.615667,88.638000,88.813333,79.568000,88.662333,88.210667,87.773000,88.814000,82.956000
WAVEFORM,80.289000,80.289000,78.770333,80.053667,80.280667,80.268000,68.334667,80.251667,80.289000,80.225000,80.037667,80.285333


## Table Abrupt NaiveBayes 20K

In [11]:
classificator = "bayes.NaiveBayes"
drift_size = 20000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,68.598167,67.612500,66.596833,66.416333,69.048167,68.534167,58.167667,68.999167,68.613833,68.160333,68.549000,67.701667
AGRAW2,87.566500,86.917500,86.509500,85.274500,87.782333,87.823833,79.029833,87.811667,87.152833,87.392167,88.033667,86.200833
LED,74.605000,74.605000,69.414000,74.583833,74.605000,72.567833,32.251500,74.605000,73.883000,56.772500,70.533167,65.495333
MIXED,91.023000,90.336000,89.575833,90.099500,91.312333,91.402667,77.775000,91.250167,90.994000,90.502333,91.162333,87.847833
RANDOM_RBF,88.100667,88.474333,85.999167,87.071000,88.707833,88.783000,50.140000,88.633167,88.460000,71.833833,87.133333,77.749667
SINE,89.003000,85.031667,88.474500,88.077500,89.196333,89.295167,79.808000,89.093833,88.758333,88.609833,89.267833,86.324833
WAVEFORM,80.334667,80.334667,78.922000,80.209000,80.329333,80.326500,68.381667,80.343667,80.332667,80.308333,80.136333,80.330500


## Table Abrupt NaiveBayes 50K

In [12]:
classificator = "bayes.NaiveBayes"
drift_size = 50000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,69.832733,67.762933,66.986667,67.363933,69.826733,69.093400,58.062400,69.865333,69.616667,69.264333,69.302067,69.602267
AGRAW2,88.541867,87.858067,86.746667,85.987200,88.516333,88.442867,79.109200,88.502600,88.153600,88.375533,88.387400,87.946000
LED,74.344800,74.344800,69.096733,74.338867,74.202200,72.209800,32.056333,74.254067,73.945200,57.747267,70.600933,71.156200
MIXED,91.656667,90.684333,89.902867,90.014467,91.715267,91.804933,77.934400,91.654800,91.486467,91.473667,91.506200,90.367133
RANDOM_RBF,89.645000,89.564600,86.404067,87.969200,89.811000,89.919200,50.204733,89.789133,89.666933,72.189000,88.505400,81.658133
SINE,89.355200,85.140933,88.432400,88.041667,89.354467,89.469400,80.014933,89.280200,89.057800,89.195600,89.320600,88.250600
WAVEFORM,80.400000,80.400000,79.140333,80.373800,80.399400,80.350533,68.467133,80.393467,80.395800,80.385333,80.243533,80.398600


## Table Abrupt NaiveBayes 100K

In [13]:
classificator = "bayes.NaiveBayes"
drift_size = 100000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.278767,67.867567,67.061267,67.948233,70.276767,69.400200,58.087300,70.274633,69.959333,69.752333,69.564033,70.212767
AGRAW2,88.906433,87.549600,86.757833,86.250233,88.848500,88.707867,79.058100,88.814200,88.585867,88.795200,88.503233,88.613800
LED,74.317467,74.317467,69.263133,74.312500,74.218600,72.387833,32.071733,74.116300,73.984067,58.770433,70.074000,72.735100
MIXED,91.866600,90.608300,89.855033,89.740033,91.844700,91.942500,77.972900,91.825433,91.712400,91.743000,91.573000,91.234167
RANDOM_RBF,90.245867,89.752633,86.526167,88.578767,90.301167,90.385700,50.262367,90.287967,90.206733,72.549500,88.789233,84.280167
SINE,89.404600,82.818367,88.408200,88.050433,89.316400,89.462000,80.013833,89.373467,89.190467,89.278167,89.317767,88.846667
WAVEFORM,80.417600,80.419733,79.177133,80.412033,80.418733,80.394300,68.451700,80.428567,80.415933,80.398100,80.299533,80.418933


## Table Abrupt NaiveBayes 500K

In [14]:
classificator = "bayes.NaiveBayes"
drift_size = 500000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.69068,68.11760,67.09644,68.37030,70.61626,69.58850,58.10496,70.62170,70.33906,70.41014,69.69536,70.66708
AGRAW2,89.34564,88.60918,86.84728,86.68052,89.29722,89.02072,79.08434,89.23788,89.11070,89.25210,88.66292,89.28378
LED,74.07494,74.10654,69.23864,74.10590,74.00964,72.50628,32.13844,73.81776,73.84808,65.13190,70.12876,73.82534
MIXED,92.04850,90.45936,89.95416,90.66508,92.01242,92.06362,78.03600,92.00830,91.97122,92.00466,91.63010,91.92460
RANDOM_RBF,90.76354,90.23904,86.65542,88.70232,90.60518,90.79502,50.23110,90.72660,90.63302,73.52316,89.14422,89.18306
SINE,89.40398,82.27622,88.44446,84.53798,89.34066,89.41134,80.01384,89.43522,89.30196,89.36210,89.38952,89.30290
WAVEFORM,80.44634,80.44634,79.22972,80.44340,80.44652,80.39514,68.46380,80.44718,80.45112,80.43862,80.29862,80.44634


## Table Abrupt NaiveBayes 1M

In [15]:
classificator = "bayes.NaiveBayes"
drift_size = 1000000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                     "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.75968,68.70482,67.09935,68.47945,70.68788,69.65189,58.08772,70.70616,70.46495,70.51735,69.77137,70.74140
AGRAW2,89.39134,88.21051,86.82117,86.75809,89.34224,89.02198,79.07748,89.26711,89.14965,89.30240,88.65120,89.35881
LED,74.06937,74.06937,69.27354,74.06907,73.99151,72.53909,32.14839,73.86471,73.87173,63.29537,70.09169,73.89805
MIXED,92.08354,90.20928,89.96825,87.56309,92.06546,92.09128,78.03526,92.03799,92.00401,92.04781,91.65645,92.02166
RANDOM_RBF,90.86955,89.64490,86.66704,88.97805,90.84770,90.87784,50.26228,90.78519,90.75843,73.44809,89.29402,90.10112
SINE,89.45361,81.30600,88.45821,86.66229,89.40483,89.45481,80.02735,89.50843,89.33680,89.41922,89.39453,89.40365
WAVEFORM,80.45072,80.45039,79.21103,80.44879,80.45504,80.40267,68.44628,80.45307,80.45214,80.44758,80.28985,80.45206


## Table Abrupt NaiveBayes 2M

In [16]:
classificator = "bayes.NaiveBayes"
drift_size = 2000000
type_drift = "Abrupt"

dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                     "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

# dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
#                     "abrupt_sine": "SINE"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.800620,68.106905,67.106550,68.506845,70.716230,69.686825,58.107180,70.705625,70.625170,70.575740,69.769590,70.792250
AGRAW2,89.428305,88.106615,86.825790,86.827665,89.384785,89.045310,79.074390,89.291510,89.237745,89.340580,88.659815,89.409900
LED,74.066020,74.066020,69.330835,74.065805,73.977430,72.546540,32.150130,73.888980,73.971410,64.026195,70.175920,73.778425
MIXED,92.062560,90.181935,89.962840,88.914010,92.040060,92.066730,78.013030,92.017130,91.998775,92.036280,91.646705,92.031740
RANDOM_RBF,90.910555,89.801900,86.651280,88.756185,90.874590,90.906300,50.253375,90.788090,90.769260,73.369565,89.312220,90.470700
SINE,89.475315,84.916475,88.460100,86.461175,89.453020,89.476605,80.040290,89.535310,89.398410,89.457130,89.417235,89.450445
WAVEFORM,80.478750,80.478475,79.232165,80.478065,80.480700,80.441895,68.456525,80.480410,80.483880,80.479080,80.331135,80.478610


## Table Abrupt Hoeffding Tree 10K

In [17]:
classificator = "trees.HoeffdingTree"
drift_size = 10000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,67.071667,66.438333,66.066333,65.625333,68.189000,67.962667,59.358667,68.236667,67.475000,65.893333,67.004667,65.986667
AGRAW2,85.992667,85.056667,85.855000,83.799000,86.867667,86.980000,78.684000,86.664667,85.759000,85.896333,86.872333,83.642000
LED,74.795000,74.795000,69.841000,74.748000,74.764000,73.813667,31.425667,74.795000,73.563333,55.713667,67.918000,62.367667
MIXED,89.957667,90.300667,89.407667,89.273000,90.824333,90.854000,76.906000,90.674333,90.344000,89.514333,90.795667,83.753000
RANDOM_RBF,85.585333,86.700333,84.972333,85.546667,86.926667,86.992000,49.038667,86.865667,86.666333,70.087333,83.870333,75.583667
SINE,89.374333,89.208667,88.126667,87.645667,90.080000,90.099667,78.922667,89.664000,89.544667,88.713000,89.677667,84.125000
WAVEFORM,79.895000,79.906333,78.748000,79.738333,79.931000,80.008333,67.627667,79.955000,79.906333,79.910667,79.953333,79.894000


## Table Abrupt Hoeffding Tree 20K

In [18]:
classificator = "trees.HoeffdingTree"
drift_size = 20000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,72.035500,70.840500,67.003500,68.575667,72.445333,71.107333,59.333333,72.266000,72.119333,67.664000,68.927833,67.967000
AGRAW2,87.656333,86.896000,86.299333,85.014833,88.043500,88.078500,78.818833,87.771167,87.096500,87.136167,87.979833,86.340833
LED,74.603000,74.603000,69.368500,74.581500,74.603000,72.547667,31.395167,74.603000,73.880167,56.107000,69.743000,65.202333
MIXED,90.405667,88.860167,89.538500,89.557500,90.460167,90.821333,77.094167,90.864667,90.630500,90.231167,90.871667,87.929000
RANDOM_RBF,88.059000,88.469000,85.953333,86.964000,88.693167,88.770500,49.165333,88.619667,88.455833,70.818167,86.804833,77.471667
SINE,91.272500,90.575333,88.608500,88.867500,91.585333,91.609000,79.146833,91.045833,91.185333,89.809000,90.827000,88.481167
WAVEFORM,79.782333,79.828833,78.901833,79.718833,79.868667,80.109500,67.675167,80.043833,79.949333,79.826167,80.074667,79.758500


## Table Abrupt Hoeffding Tree 50K

In [19]:
classificator = "trees.HoeffdingTree"
drift_size = 50000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,77.333200,76.497800,67.975533,72.563800,77.499667,75.528533,59.307000,77.335867,77.291400,68.507267,70.562467,71.563133
AGRAW2,90.298400,89.944800,86.535800,87.668467,90.323400,89.575867,78.790933,90.160400,90.007933,87.930600,89.087000,88.921733
LED,73.941000,73.854800,69.083933,73.655133,73.898667,72.191067,31.175600,73.941467,73.561933,57.550067,69.929733,69.743200
MIXED,92.133867,91.655000,89.856333,90.488600,92.202467,92.242667,77.248333,91.691800,91.886867,91.020800,91.200533,90.640600
RANDOM_RBF,89.624000,89.563400,86.397000,88.037867,89.811133,89.913600,49.243733,89.785667,89.665667,71.262933,88.254000,80.760067
SINE,93.178467,92.749933,88.684733,90.244867,93.285933,93.305333,79.380933,92.821800,93.005467,90.503933,92.110133,91.977133
WAVEFORM,80.043667,80.076667,79.124733,80.052867,80.094333,80.132000,67.748533,80.198667,80.083067,79.959133,80.208400,80.067933


## Table Abrupt Hoeffding Tree 100K

In [20]:
classificator = "trees.HoeffdingTree"
drift_size = 100000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}



df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,80.374733,80.235300,69.405300,73.715667,80.140367,77.733167,59.304433,80.134333,80.284467,68.764067,71.730033,73.302467
AGRAW2,92.197200,91.987767,86.562167,89.972800,92.076567,90.886700,78.767867,92.049067,91.987567,88.164067,90.066000,90.567933
LED,73.863300,73.677733,69.228400,73.807267,73.830933,72.370667,31.189200,73.853233,73.576200,57.634367,69.771533,70.916767
MIXED,93.078067,92.867300,89.791600,91.481933,93.151567,93.201600,77.287700,92.528767,92.950800,91.395100,91.252800,92.453133
RANDOM_RBF,90.247467,89.839133,86.513133,88.454600,90.303000,90.383067,49.308200,90.286767,90.206167,71.427200,88.591100,83.537467
SINE,94.178933,93.918767,88.731333,91.913433,94.216600,94.260567,79.368667,93.969567,94.019000,90.737100,92.738800,93.553667
WAVEFORM,81.364533,81.329633,79.164067,81.130867,81.024633,80.144167,67.739967,80.396300,80.466733,79.915333,80.258267,81.357767


## Table Abrupt Hoeffding Tree 500K

In [21]:
classificator = "trees.HoeffdingTree"
drift_size = 500000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}



df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,84.65794,84.36160,69.94126,80.99544,84.13120,80.53794,59.30378,83.44914,83.98706,69.08398,73.43544,77.02490
AGRAW2,94.94620,94.84970,86.61738,93.42858,94.87234,94.20762,78.78304,94.22054,94.51074,88.48504,91.45564,92.56522
LED,73.62642,73.81264,69.21604,74.00762,73.59590,72.49172,31.25922,73.65604,73.45618,58.20198,69.83330,71.22636
MIXED,94.92076,94.79206,89.90134,93.69658,94.93008,94.93412,77.35282,94.22232,94.39082,91.92484,91.50580,94.75488
RANDOM_RBF,90.18262,93.67242,86.64146,93.39356,91.69090,90.35794,49.27202,91.54730,92.24252,71.85500,88.98892,84.98534
SINE,96.82722,96.69294,88.73640,95.61464,96.82214,96.86296,79.36110,96.10504,96.32446,90.96796,93.52014,96.56292
WAVEFORM,83.36834,83.38118,79.22346,83.29200,82.70286,80.18652,67.75172,80.34068,81.12668,79.85190,80.24798,83.32362


## Table Abrupt Hoeffding Tree 1M

In [22]:
classificator = "trees.HoeffdingTree"
drift_size = 1000000
type_drift = "Abrupt"
dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,85.62021,85.61276,70.03307,81.91679,85.57248,81.36807,59.27914,84.00029,84.52702,69.08668,73.33016,77.51459
AGRAW2,95.59142,95.51853,86.60209,94.30087,95.42110,95.18665,78.77962,94.51155,94.64106,88.60867,91.95465,93.07200
LED,73.63478,74.00479,69.25716,74.08841,73.62416,72.52443,31.27178,73.65149,73.47984,58.93002,69.77890,71.30066
MIXED,95.97479,95.89326,89.92398,94.96484,95.97295,95.99016,77.35524,94.42605,94.85907,92.05342,91.48592,95.91115
RANDOM_RBF,90.19116,94.94890,86.65503,94.32844,92.78229,90.37218,49.30096,91.55185,92.52935,71.65137,89.13959,85.42130
SINE,97.74146,97.63867,88.74262,96.98312,97.76433,97.73487,79.37462,96.45150,96.81352,91.00748,93.74676,97.58569
WAVEFORM,83.92729,83.97001,79.20109,83.91260,83.34960,80.15868,67.73807,80.46711,81.11610,79.97168,80.25084,83.88854


## Table Abrupt Hoeffding Tree 2M

In [23]:
classificator = "trees.HoeffdingTree"
drift_size = 2000000
type_drift = "Abrupt"

dict_drift_type = {"abrupt_agraw1": "AGRAW1", "abrupt_agraw2": "AGRAW2", "abrupt_led": "LED", "abrupt_mixed": "MIXED", "abrupt_randomRBF": "RANDOM_RBF",  
                    "abrupt_sine": "SINE", "abrupt_waveform": "WAVEFORM"}

df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,87.263175,87.437225,70.178815,81.925405,86.114650,81.833465,59.281535,84.766265,85.045105,69.195335,73.593530,78.423505
AGRAW2,96.006570,95.986345,86.614550,94.845660,95.910810,95.838310,78.775190,94.557040,94.868850,88.545685,91.848165,93.611165
LED,73.662790,74.083780,69.319845,74.122310,73.663315,72.532185,31.277065,73.687170,73.662320,58.801090,69.904645,70.919985
MIXED,96.895930,96.855575,89.921090,96.129310,96.918915,96.908715,77.330060,94.431320,95.027265,92.094960,91.480195,96.876045
RANDOM_RBF,90.219635,95.724755,NaN,95.180575,93.719730,90.382455,49.292865,91.780485,92.575915,71.632820,89.154715,85.657860
SINE,98.492640,98.443195,88.756350,98.054965,98.480485,98.483730,79.388370,96.643360,96.946620,90.994725,93.894250,98.412360
WAVEFORM,84.347380,84.402535,79.219325,84.376820,83.636590,80.186790,67.743325,80.256690,81.146785,79.968060,80.292750,84.159225


## Table Gradual NaiveBayes 10K

In [24]:
classificator = "bayes.NaiveBayes"
drift_size = 10000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,64.725333,65.521333,64.871667,64.390000,66.103000,66.310667,57.348667,66.551667,65.250000,65.201000,65.370667,64.374333
AGRAW2,82.677000,82.552000,81.325667,81.200667,82.975333,83.193000,74.490667,83.618667,83.023000,81.807333,82.771333,79.996000
LED,74.805333,74.805333,69.791667,74.758333,74.739000,73.696000,32.143333,74.805333,74.805333,56.743333,68.428667,65.802667
MIXED,85.379667,84.989667,84.535000,85.373000,84.868667,85.271667,73.720667,85.218000,85.244000,84.102000,84.667333,84.401333
RANDOM_RBF,81.218333,83.343667,81.110333,82.941333,83.159000,82.932333,47.986333,83.232333,83.368333,70.278000,80.178000,74.458000
SINE,84.711333,84.387333,83.989667,84.516667,84.446333,84.677333,75.668333,84.600667,84.662333,83.301667,83.977333,83.400000
WAVEFORM,80.292000,80.292000,78.745667,80.010667,80.258667,80.241667,68.348667,80.272667,80.292000,80.276000,80.059333,80.288333


## Table Gradual NaiveBayes 20K

In [25]:
classificator = "bayes.NaiveBayes"
drift_size = 20000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,67.724167,67.407500,65.831667,65.754333,68.155500,67.655333,57.644500,68.267333,67.803000,67.343833,67.580833,66.488333
AGRAW2,85.517833,85.709833,83.860333,84.554500,85.845167,85.624167,76.436167,85.983000,85.733667,84.899833,85.399333,84.231000
LED,74.613167,74.613167,68.918667,74.594500,74.592000,72.939833,32.264667,74.613167,74.613167,56.982500,69.917333,68.692167
MIXED,87.842667,87.678333,86.528833,87.929333,87.612667,87.864833,75.320333,88.004167,87.952000,86.785667,87.150667,87.520500
RANDOM_RBF,85.394833,86.333500,83.826833,86.044833,86.240333,86.079833,48.919833,86.294000,86.346833,71.133833,84.324500,77.651500
SINE,86.729000,86.766500,85.961667,86.610333,86.754000,86.697000,77.277000,86.848333,86.846667,85.494000,85.919833,85.969500
WAVEFORM,80.340333,80.340333,78.929500,80.212667,80.331500,80.319167,68.373833,80.346667,80.343333,80.317667,80.156167,80.335167


## Table Gradual NaiveBayes 50K

In [26]:
classificator = "bayes.NaiveBayes"
drift_size = 50000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,69.549533,68.255800,66.723067,67.145667,69.618000,68.841733,57.926133,69.642067,69.054533,68.909733,68.949000,69.409467
AGRAW2,87.770400,87.440400,85.762400,85.966667,87.560667,87.579467,78.168400,87.860933,87.573400,87.503867,87.429867,87.404800
LED,74.342000,74.342000,69.143867,74.335867,74.199800,72.212800,32.062933,74.255200,74.301867,57.157800,70.459533,71.422667
MIXED,90.588133,90.563400,88.868533,90.274000,90.568467,90.563267,77.070333,90.594867,90.583533,90.235200,90.127600,90.469733
RANDOM_RBF,88.739800,89.061733,85.730400,88.231200,89.020000,88.976000,49.773400,89.070933,89.058333,72.119133,87.584467,82.114600
SINE,88.720467,88.801800,87.629933,88.099800,88.847400,88.848933,79.161933,88.878067,88.848667,88.104933,88.260133,88.382933
WAVEFORM,80.399533,80.399533,79.150000,80.376267,80.393467,80.343867,68.458800,80.396067,80.396600,80.389333,80.235200,80.398200


## Table Gradual NaiveBayes 100K

In [27]:
classificator = "bayes.NaiveBayes"
drift_size = 100000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.163167,68.424633,66.918000,67.892333,70.169800,69.254300,58.011600,70.186233,69.915267,69.614467,69.451900,70.104933
AGRAW2,88.492333,87.983400,86.328300,86.163767,88.472233,88.219000,78.617267,88.487333,88.341167,88.343500,87.873200,88.332833
LED,74.320567,74.320567,69.217633,74.316033,74.238867,72.343000,32.080533,74.117367,74.150033,59.636800,70.203800,73.223900
MIXED,91.334467,91.279200,89.355700,90.636567,91.312067,91.339133,77.572267,91.353433,91.340500,91.144200,90.933233,91.285567
RANDOM_RBF,89.791267,89.930700,86.171000,88.777200,89.917100,89.906400,50.015800,89.951667,89.929600,72.342733,88.372367,83.744967
SINE,89.122967,89.008833,88.001333,88.177567,89.212533,89.233733,79.634200,89.255167,89.186433,88.784000,88.806700,88.924067
WAVEFORM,80.416700,80.417333,79.173500,80.409467,80.416167,80.389833,68.462833,80.420300,80.417867,80.404133,80.305800,80.416367


## Table Gradual NaiveBayes 500K

In [28]:
classificator = "bayes.NaiveBayes"
drift_size = 500000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                   "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.66098,68.96492,67.07866,68.35260,70.56590,69.56402,58.12124,70.58288,70.44708,70.39294,69.68004,70.64066
AGRAW2,89.24634,88.82542,86.76880,86.70834,89.20514,88.89716,78.97864,89.13418,89.02848,89.15866,88.54456,89.24240
LED,74.08926,74.10630,69.23164,74.10534,74.00802,72.51826,32.14370,73.85594,73.99104,63.69346,70.11282,73.85302
MIXED,91.93446,91.86916,89.85598,90.67902,91.94012,91.93290,77.94028,91.90612,91.89308,91.88804,91.51016,91.92122
RANDOM_RBF,90.67762,90.28274,86.58164,88.74200,90.69780,90.71134,50.21386,90.67666,90.60634,73.05620,89.05156,89.24050
SINE,89.35590,86.31444,88.35624,87.02468,89.33942,89.37844,79.95060,89.40900,89.23072,89.25422,89.28778,89.30848
WAVEFORM,80.44566,80.44566,79.22954,80.44282,80.44598,80.39456,68.45056,80.44254,80.45022,80.44336,80.29806,80.44634


## Table Gradual NaiveBayes 1M

In [29]:
classificator = "bayes.NaiveBayes"
drift_size = 1000000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                   "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.74243,68.83349,67.08299,68.46260,70.67991,69.63662,58.08734,70.63373,70.49782,70.49006,69.74864,70.72666
AGRAW2,89.33771,87.93036,86.77348,86.73244,89.25954,88.94647,79.02960,89.21247,89.12166,89.24588,88.59277,89.34100
LED,74.06871,74.06871,69.28507,74.06822,73.99063,72.53204,32.12282,73.86816,73.99665,63.40762,70.09986,73.92408
MIXED,92.02727,91.87631,89.92086,88.65357,92.02656,92.02687,77.98745,91.98133,91.98167,91.99255,91.58758,92.01881
RANDOM_RBF,90.81141,90.72899,86.62638,89.42851,90.81093,90.82837,50.23990,90.75627,90.70147,73.60852,89.25057,90.07843
SINE,89.43297,82.98712,88.41546,84.47796,89.40783,89.45950,79.99482,89.48083,89.35086,89.37810,89.33852,89.40195
WAVEFORM,80.45055,80.45025,79.20939,80.44879,80.45497,80.40339,68.43714,80.44674,80.45243,80.45082,80.29248,80.45145


## Table Gradual NaiveBayes 2M

In [30]:
classificator = "bayes.NaiveBayes"
drift_size = 2000000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                   "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.792350,68.406605,67.099130,68.577825,70.719375,69.679895,58.096545,70.705390,70.612475,70.559840,69.756380,70.785895
AGRAW2,89.390920,88.005240,86.803780,86.875255,89.339950,88.993940,79.048365,89.260960,89.215850,89.316485,88.637295,89.393335
LED,74.065775,74.065775,69.332175,74.065555,73.980505,72.548375,32.154590,73.897125,74.018390,62.066935,70.166370,73.937890
MIXED,92.034310,90.400195,89.935465,88.692105,92.022800,92.032465,77.990925,91.988895,91.980370,92.010980,91.613270,92.030055
RANDOM_RBF,90.886765,89.093075,86.634450,89.009425,90.863320,90.885685,50.256405,90.747760,90.749980,73.393515,89.293505,90.579950
SINE,89.474115,81.701095,88.438930,87.877640,89.457400,89.487185,80.026170,89.517860,89.389285,89.431115,89.387335,89.450360
WAVEFORM,80.478615,80.478340,79.231310,80.478010,80.480555,80.441475,68.457030,80.481550,80.481065,80.476800,80.331525,80.479510


## Table Gradual HoeffdingTree 10K

In [31]:
classificator = "trees.HoeffdingTree"
drift_size = 10000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,65.483667,65.850667,64.567333,64.799333,66.188667,66.243000,58.410000,66.713333,65.723000,64.907000,65.438667,64.826667
AGRAW2,83.357333,83.500000,83.063333,81.614000,83.986667,84.110333,76.234333,84.348333,83.568000,83.231000,83.819333,82.611667
LED,74.801333,74.801333,69.740000,74.753667,74.735000,73.682333,31.297333,74.801333,74.801333,56.469333,68.073333,64.491333
MIXED,85.351333,84.817667,84.388333,85.274667,84.712667,85.148000,72.973000,85.045333,85.110667,83.975333,84.516333,84.200000
RANDOM_RBF,81.245667,83.333333,81.059333,82.891000,83.113000,82.895333,46.986667,83.223667,83.362000,69.238667,79.104000,74.208000
SINE,85.278000,84.943000,84.210333,84.794000,85.080667,85.313333,74.919000,85.170000,85.248667,83.508000,84.262000,83.974667
WAVEFORM,79.916333,79.918667,78.730667,79.701333,79.944000,79.977000,67.641000,79.950333,79.913333,79.957000,79.956333,79.937667


## Table Gradual HoeffdingTree 20K

In [32]:
classificator = "trees.HoeffdingTree"
drift_size = 20000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,70.384333,70.487333,65.862833,67.903333,70.338167,69.607500,58.753000,70.954667,70.766000,66.831000,67.670833,66.821833
AGRAW2,85.978500,85.621000,84.663167,83.988333,86.226500,86.158667,77.215333,86.316667,85.576500,85.409167,86.068333,85.537000
LED,74.611167,74.611167,68.921667,74.592167,74.590000,72.923500,31.392000,74.611167,74.611167,56.363000,69.984500,68.781833
MIXED,87.174000,87.085833,86.321000,87.486333,86.998333,87.152000,74.621333,87.377167,87.288333,86.684167,86.985833,87.268167
RANDOM_RBF,85.389500,86.328833,83.854333,86.039667,86.230833,86.063333,47.961500,86.285000,86.341667,70.079833,83.924667,77.317000
SINE,88.223667,88.020167,86.157500,87.589833,88.024000,88.133500,76.575333,88.071500,88.180167,86.117833,87.027167,87.651000
WAVEFORM,79.660333,79.741167,78.910667,79.696500,79.781167,80.058667,67.653000,80.026833,79.915167,79.895333,80.076667,79.612500


## Table Gradual HoeffdingTree 50K

In [33]:
classificator = "trees.HoeffdingTree"
drift_size = 50000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,76.363667,75.774467,68.280000,72.583133,76.190733,74.362067,59.100333,76.281400,75.736067,68.234333,69.969733,71.040333
AGRAW2,89.706800,89.603600,85.966933,87.586000,89.677067,88.830000,78.256933,89.703533,89.578867,87.279667,88.433333,88.703533
LED,73.957467,73.860467,69.145400,73.651333,73.912867,72.194933,31.189267,73.946800,73.854867,57.923933,69.754933,69.631867
MIXED,91.042267,91.015733,88.787467,90.470533,90.980133,90.993200,76.375733,90.897600,91.011467,89.804067,89.839867,90.699467
RANDOM_RBF,88.740133,89.060333,85.722200,88.246200,89.007467,88.968267,48.832267,89.068533,89.056867,71.021267,87.270067,81.543200
SINE,92.100467,92.110133,87.889867,90.479067,92.147733,92.168933,78.507133,91.945133,92.126000,89.417800,90.804000,91.883733
WAVEFORM,80.091000,80.146533,79.135200,80.039800,80.070000,80.188933,67.741667,80.173600,80.098067,79.926000,80.198600,80.071333


## Table Gradual HoeffdingTree 100K

In [34]:
classificator = "trees.HoeffdingTree"
drift_size = 100000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,79.584433,79.528267,69.124600,74.043100,79.274633,77.225633,59.190467,79.841767,79.019233,68.655000,71.453767,73.209700
AGRAW2,91.876900,91.778167,86.334700,89.887933,91.752400,90.557067,78.544833,91.853000,91.802800,87.915733,89.882267,90.513033
LED,73.870733,73.676567,69.174700,73.814800,73.846133,72.325800,31.198967,73.854300,73.740333,57.860367,69.620667,70.627233
MIXED,92.524767,92.540567,89.265067,91.588467,92.511500,92.569200,76.886133,92.444467,92.573633,90.758533,90.721400,92.424233
RANDOM_RBF,89.777600,89.930033,86.164300,88.832100,89.913767,89.902767,49.056033,89.951033,89.928767,71.336700,88.172833,83.149333
SINE,93.628067,93.642867,88.357100,92.158633,93.631333,93.703833,78.972933,93.598867,93.686400,90.105433,92.206200,93.521433
WAVEFORM,81.370267,81.392533,79.160433,81.203133,81.100133,80.163167,67.754400,80.207367,80.516667,79.918433,80.258233,81.383267


## Table Gradual HoeffdingTree 500K

In [35]:
classificator = "trees.HoeffdingTree"
drift_size = 500000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,84.44918,84.71580,69.81624,80.18920,83.91736,80.41358,59.30704,84.34000,84.20208,68.97860,73.32046,77.09582
AGRAW2,94.80338,94.81180,86.56902,93.47528,94.75560,94.09270,78.71908,94.23928,94.41580,88.48706,91.41422,92.50116
LED,73.62640,73.81630,69.20840,74.00748,73.62034,72.50442,31.26922,73.66366,73.51224,57.99974,69.93952,70.91772
MIXED,94.79278,94.82198,89.79700,93.70288,94.76202,94.80734,77.25082,94.05394,94.40166,91.83158,91.40654,94.71384
RANDOM_RBF,90.08612,93.61634,86.57106,93.37532,91.44764,90.26828,49.25562,91.61730,92.29448,71.56840,88.89368,85.44852
SINE,96.59146,96.59432,88.66566,95.49306,96.49334,96.56588,79.29500,96.15070,96.30760,90.83204,93.62930,96.54242
WAVEFORM,83.36864,83.38246,79.22368,83.29442,82.80082,80.10628,67.74450,80.44332,80.97908,79.95242,80.24520,83.32302


## Table Gradual HoeffdingTree 1M

In [36]:
classificator = "trees.HoeffdingTree"
drift_size = 1000000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,84.91885,86.12775,70.05295,81.24670,84.15530,81.30401,59.26952,84.40842,84.90432,69.09120,73.42338,78.10496
AGRAW2,95.49324,95.49957,86.57704,94.31995,95.39612,95.15883,78.74840,94.44865,94.64009,88.51612,91.96605,93.01831
LED,73.63381,74.00573,69.26873,74.08901,73.64248,72.51714,31.24854,73.66634,73.61110,58.79509,69.84070,71.15876
MIXED,95.91989,95.91995,89.86778,95.05116,95.88547,95.89175,77.30935,94.40478,94.83667,91.90192,91.42715,95.85491
RANDOM_RBF,90.15502,94.70258,86.61774,94.44684,92.14522,90.50788,49.28092,91.66025,92.26319,71.64677,89.10180,85.45251
SINE,97.61321,97.60111,88.70750,96.95738,97.55654,97.63077,79.34232,96.69601,96.94927,90.92721,93.68784,97.57755
WAVEFORM,83.92935,83.96999,79.19933,83.91361,83.20800,80.12905,67.72656,80.42500,80.99642,79.93968,80.25343,83.88909


## Table Gradual HoeffdingTree 2M

In [37]:
classificator = "trees.HoeffdingTree"
drift_size = 2000000
type_drift = "Gradual"
dict_drift_type = {"gradual_agraw1": "AGRAW1", "gradual_agraw2": "AGRAW2", "gradual_led": "LED", 
                    "gradual_mixed": "MIXED", "gradual_randomRBF": "RANDOM_RBF", "gradual_sine": "SINE", "gradual_waveform": "WAVEFORM"}


df_final_result = pd.DataFrame()
for drift_key, drift_value in dict_drift_type.items():    

    tuple_list_result = list()
    for drift in drift_detectors_params:
        files_path = glob.glob(f'{drift_key}/{classificator}_{drift["id"]}_{drift_size}_*_{drift_value}_{type_drift}.csv')
        #print(files_path)
        tuple_result = avg_last_execution(files_path, drift["id"], drift_value)
        tuple_list_result.append(tuple_result)
                          
    df_result = pd.DataFrame(tuple_list_result, columns=['avg', 'std', 'drift_detector', 'generator'])
    df_final_result = pd.concat([df_final_result, df_result])

df_result_pivot = df_final_result.reset_index().pivot(index="generator", columns="drift_detector", values="avg")
df_result_pivot

drift_detector,ADWIN,DDM,ECDD,EDDM,HDDM_A_Test,HDDM_W_Test,NoChangeDetector,RDDM,RDDM_30,SEED,STEPD,SeqDrift2
generator,,,,,,,,,,,,
AGRAW1,87.057475,87.192915,70.175755,82.097310,86.270285,81.885110,59.289585,84.425550,85.242795,69.112265,73.658225,79.001935
AGRAW2,95.952395,95.963145,86.602045,94.859180,95.876350,95.813070,78.761375,94.493120,94.873650,88.531980,91.829600,93.585895
LED,73.661590,74.083775,69.321305,74.122585,73.660860,72.534040,31.283115,73.673540,73.815880,58.530160,69.895155,71.071575
MIXED,96.853700,96.912290,89.891040,96.146855,96.877565,96.883895,77.308240,94.435510,94.930150,92.009690,91.444325,96.837910
RANDOM_RBF,90.195830,95.686025,86.624920,95.198295,93.910645,90.360090,49.292555,91.732130,92.684095,71.720420,89.139410,85.609500
SINE,98.404450,98.415715,88.747110,98.058145,98.364365,98.420605,79.372565,96.641735,97.118585,90.989870,93.856145,98.399140
WAVEFORM,84.344885,84.402320,79.218410,84.377055,83.602720,80.147565,67.746465,80.372580,81.410295,79.983955,80.292780,84.170195
